<a href="https://colab.research.google.com/github/GLAScorpion/CorsoIA/blob/main/Intelligenza_Artificiale_Lab6_extra_exercises.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---

## EXERCISE1: Burglary network

Implement the *Prior Sampling* algorithm to do approximate inference on last week's Burglary network.

<img src='https://drive.google.com/uc?id=11xqbchd4TCFSzdLVxNgr4SoeGfrSoqGO'>

Verify that the algorithm can correctly approximate the probability $P(j, m, a, \neg b, \neg e) = 0.00063$

Try different numbers of samples (e.g. $N = 10, 100, 1000, 10000$) and compare the results.

---

In [ ]:
import numpy as np
import random as rnd

t, f = 0, 1

def samplegen(Pdist, Parents = []):
	assert len(Parents) < len(Pdist.shape)
	if rnd.random() < Pdist[t][tuple(Parents)]:
		return t
	return f

P_B = np.array([0.001,0.999])
P_E = np.array([0.002,0.998])
P_A_BE = np.array([[[0.95,0.94],[0.29,0.001]],[[0.05,0.06],[0.71,0.999]]])
P_J_A = np.array([[0.90,0.05],[0.10,0.95]])
P_M_A = np.array([[0.70,0.01],[0.3,0.99]])

var = ['B','E','A','J','M']

prd = {'B':P_B,'E':P_E,'A':P_A_BE,'J':P_J_A,'M':P_M_A}

par = {'B':[],'E':[],'A':['B','E'],'J':['A'],'M':['A']}

val = {'B':f,'E':f,'A':f,'J':f,'M':f}

def parents(X):
	return [val[i] for i in par[X]]

def event_gen(sample_num = 1000):
  event = []
  for n in range(sample_num):
    for x in var:
      val[x] = samplegen(prd[x], parents(x))
    event.append(['f' if val[x] else 't' for x in var])
  return event

for n in range(1,7):
  val = {'B':f,'E':f,'A':f,'J':f,'M':f}
  num = pow(10,n)
  event = event_gen(num)
  prob = event.count(['f','f','t','t','t']) / len(event)
  print( "P(j,m,a,¬b,¬e) con " + str(num) + " sample vale: " + str(prob))


P(j,m,a,¬b,¬e) con 10 sample vale: 0.0
P(j,m,a,¬b,¬e) con 100 sample vale: 0.0
P(j,m,a,¬b,¬e) con 1000 sample vale: 0.001
P(j,m,a,¬b,¬e) con 10000 sample vale: 0.0004
P(j,m,a,¬b,¬e) con 100000 sample vale: 0.00064
P(j,m,a,¬b,¬e) con 1000000 sample vale: 0.000636


---

## EXERCISE2: Pomegranate for Day 2

Use <tt>pomegranate</tt> to calculate the filtered probability of rain on Day 2 when we see an umbrella on Day 1 and Day 2.

What is the filtered probability of rain on Day 2 when we see <tt>not umbrella</tt> on Day 1?

How about if we just have no information about Umbrellas on Day 1 (i.e., only that rain on Day 2)?

In [1]:
!pip install numpy
!pip install pomegranate==0.14.7
from pomegranate import *

# Variables are RainN and UmbrellaN+1 for N = 0, 1, ...
# We have a prior for Rain0, two values 'y'es and 'n'o:
Rain0   = DiscreteDistribution({'y': 0.5, 'n': 0.5})

# Transition model
#
# Conditional distribution relating RainN and RainN+1. Notation for
# the conditional probability table is:
#
# [ 'RainN', 'RainN+1', <probability>]
#
# for the conditional value P(Sprinkler|Cloudy). Note that we have to
# repeat the transition model for each pair of states
Rain1 = ConditionalProbabilityTable(
        [['y', 'y', 0.7],
         ['y', 'n', 0.3],
         ['n', 'y', 0.3],
         ['n', 'n', 0.7]], [Rain0])

Rain2 = ConditionalProbabilityTable(
        [['y', 'y', 0.7],
         ['y', 'n', 0.3],
         ['n', 'y', 0.3],
         ['n', 'n', 0.7]], [Rain1])

# Sensor model
#
# Conditional distribution relating Rain and Umbrella:
#
# [ 'Umbrella', 'Rain', <probability>]
#
# for the conditional value P(Sprinkler|Cloudy). Values for Umbrella are 'y'es and 'n'o.
# Again we have to enter the table for each day.
Umbrella1 = ConditionalProbabilityTable(
        [['y', 'y', 0.9],
         ['y', 'n', 0.1],
         ['n', 'y', 0.2],
         ['n', 'n', 0.8]], [Rain1])

Umbrella2 = ConditionalProbabilityTable(
        [['y', 'y', 0.9],
         ['y', 'n', 0.1],
         ['n', 'y', 0.2],
         ['n', 'n', 0.8]], [Rain2])
#
# The whole network has five nodes:
s1 = Node(Rain0, name="Rain0")
s2 = Node(Rain1, name="Rain1")
s3 = Node(Umbrella1, name="Umbrella1")
s4 = Node(Rain2, name="Rain2")
s5 = Node(Umbrella2, name="Umbrella2")
# Create a network that includes nodes and edges between them:
model = BayesianNetwork("Umbrella Network")
model.add_states(s1, s2, s3, s4, s5)
model.add_edge(s1, s2)
model.add_edge(s2, s3)
model.add_edge(s2, s4)
model.add_edge(s4, s5)
# Fix the model structure
model.bake()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 22.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pomegranate: filename=pomegranate-0.14.7-cp310-cp310-linux_x86_64.whl size=21347293 sha256=0b049b361f808e1b007616630a2ef4b6d6e880bae51e5baeb67061e3e99a0b98
  Stored in directory: /root/.cache/pip/wheels/9d/c3/22/31ee00d1c06723f41f60e65f5867850287d946a1b2992569e6
Successfully built pomegranate
